In [1]:

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_squared_error
import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim


In [2]:
# 데이터 로드
data = pd.read_csv('C:\\AI\\diabetes.csv')

In [3]:
# 범주형 인코딩
# data['Sex'] = LabelEncoder().fit_transform(data['Sex']) 문자일 때만 사용

In [4]:
# Outcome (라벨) 제거 ( 새로운 라벨은 BMI )
data.drop(columns=['Outcome'], inplace=True)

# 입력과 타겟 분리
X = data.drop('BMI', axis=1).values
y = data['BMI'].values.astype(np.float32)

In [5]:
# 정규화
scaler = StandardScaler()
X = scaler.fit_transform(X)

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:

# TensorDataset으로 래핑
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)


In [7]:

# 회귀 모델 정의
class RegressionModel(nn.Module):
    def __init__(self):
        super(RegressionModel, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(7, 64),  # feature 개수 일치하는지 확인
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1) # 출력층 개수 확인. 회귀면 1, 이진분류여도 1
        )

    def forward(self, x):
        return self.model(x)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = RegressionModel().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [8]:

# 학습 루프
model.train()
for epoch in range(150):
    total_loss = 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        output = model(X_batch)
        loss = criterion(output, y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_loader):.4f}")


Epoch 1, Loss: 1064.4691
Epoch 2, Loss: 1019.0934
Epoch 3, Loss: 922.2535
Epoch 4, Loss: 787.4972
Epoch 5, Loss: 603.2668
Epoch 6, Loss: 408.4592
Epoch 7, Loss: 233.8477
Epoch 8, Loss: 134.2552
Epoch 9, Loss: 100.4159
Epoch 10, Loss: 86.7771
Epoch 11, Loss: 81.4215
Epoch 12, Loss: 74.7290
Epoch 13, Loss: 71.4897
Epoch 14, Loss: 69.6951
Epoch 15, Loss: 68.3967
Epoch 16, Loss: 66.6903
Epoch 17, Loss: 67.6905
Epoch 18, Loss: 63.4529
Epoch 19, Loss: 67.7508
Epoch 20, Loss: 58.5037
Epoch 21, Loss: 57.0686
Epoch 22, Loss: 55.6997
Epoch 23, Loss: 55.8995
Epoch 24, Loss: 53.6646
Epoch 25, Loss: 52.9540
Epoch 26, Loss: 51.8817
Epoch 27, Loss: 56.1195
Epoch 28, Loss: 49.7902
Epoch 29, Loss: 51.1388
Epoch 30, Loss: 49.3233
Epoch 31, Loss: 49.3528
Epoch 32, Loss: 46.1762
Epoch 33, Loss: 45.5450
Epoch 34, Loss: 52.1793
Epoch 35, Loss: 43.7056
Epoch 36, Loss: 44.1304
Epoch 37, Loss: 42.8513
Epoch 38, Loss: 42.6289
Epoch 39, Loss: 50.9376
Epoch 40, Loss: 41.4964
Epoch 41, Loss: 41.4384
Epoch 42, Loss

In [10]:

# 평가
model.eval()
preds, actuals = [], []
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch = X_batch.to(device)
        outputs = model(X_batch).cpu().numpy()
        preds.extend(outputs)
        actuals.extend(y_batch.numpy())

mse = mean_squared_error(actuals, preds)
print(f"Test MSE: {mse:.4f}")


Test MSE: 57.6594
